In [16]:
# Import dependencies for seleniun & web-scraping
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time

# Path from chromdriver
executable_path={'executable_path':'./chromedriver'}

# Browser object from splinter
browser=Browser('chrome',**executable_path,headless=False)

Scrape 1: NASA Mars New

In [17]:
# Page URL
url="https://mars.nasa.gov/news/"

# Get html from browser
browser.visit(url)

# Store list of html

html=browser.html

# Use last element of HTML list
soup=bs(html,'html.parser')

# Initial soup based on each list item
starter_soup=soup.find('li',class_="slide")

# Loop through initial soup & find all headlines, dates, & titles
headline = (starter_soup.find('div',class_="content_title")).text.strip()
date = (starter_soup.find('div',class_="list_date")).text.strip()
paragraph = (starter_soup.find('div',class_="article_teaser_body")).text.strip()

In [ ]:
# Page URL
url="https://mars.nasa.gov/news/"

# Get html from browser
browser.visit(url)

# Store list of html
htmls=[]

# Switch for while loop
run=True

# Each click reveals more headlines, leading to potential duplicates
# Work around this by saving HTML for each iteration, then work off last HTML
while run:

    try:
        browser.find_by_css('a.button').first.click()
        
    except:
        run=False    
    htmls.append(browser.html)

    

In [45]:
# Use last element of HTML list
soup=bs(htmls[-1],'html.parser')

# Initial soup based on each list item
starter_soup=soup.find_all('li',class_="slide")

# Initialize result lists for headlines, dates, & titles html
headlines_soup=[]
dates_soup=[]
paragraph_soup=[]

# Initialize result lists for scraped headlines, dates, & titles
headlines=[]
dates=[]
paragraphs=[]

# Loop through initial soup & find all headlines, dates, & titles
for item in starter_soup:
    headlines_soup.append(item.find_all('div',class_="content_title"))
    dates_soup.append(item.find_all('div',class_="list_date"))
    paragraph_soup.append(item.find_all('div',class_="article_teaser_body"))

# Append each headline to the headlines array
for line in headlines_soup:
    for l in line:
#         print(l.text.strip())
        headlines.append(l.text.strip())

# Append each date to the dates array
for line in dates_soup:
    for l in line:
#         print(l.text.strip())
        dates.append(l.text.strip())

# Append each paragraph to the paragraphs array
for line in paragraph_soup:
    for l in line:
#         print(l.text.strip())
        paragraphs.append(l.text.strip())

# Confirm arrays are of equal length
print(f"Headlines: {len(headlines)}, dates: {len(dates)}, paragraphs: {len(paragraphs)}")

Headlines: 1855, dates: 1855, paragraphs: 1855


Scrape 2: JPL Mars Space Images - Featured Image

In [7]:
# Page URL
url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

# Get visit url
browser.visit(url)

# Click on "full image" button
browser.find_by_css('a#full_image.button.fancybox').first.click()

In [9]:
# Click on "more info" button
browser.click_link_by_partial_href('/spaceimages/details')

In [10]:
# Click on image
browser.find_by_css('img.main_image').first.click()

In [11]:
# Pull browser html
html=browser.html
# Parse with BS
soup=bs(html,'html.parser')
# Find image element
image=soup.find_all('img')
# Pull source url
featured_url=image[0]['src']

Scrape 3: Mars Weather

In [19]:
# Page URL
url="https://twitter.com/marswxreport?lang=en"

# Visit url
browser.visit(url)

In [20]:
# Pull browser url
html=browser.html
# Parse with BS
soup=bs(html,'html.parser')
# Find first tweet
tweet=soup.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')

# Weather
mars_weather=tweet.text
# Remove img url
mars_weather=mars_weather.split('pic.twitter')[0]
print(mars_weather)

InSight sol 141 (2019-04-20) low -98.3ºC (-144.9ºF) high -19.7ºC (-3.5ºF)
winds from the SW at 4.7 m/s (10.6 mph) gusting to 12.9 m/s (28.8 mph)
pressure at 7.40 hPa


Scrape 4: Mars Facts

In [23]:
# Page URL
url="https://space-facts.com/mars/"

# Use pandas to pull in tables
import pandas as pd
mars_facts = pd.read_html(url)[0]

In [24]:
# Convert table to HTML using pandas
facts_html = mars_facts.to_html()
# Strip our newlines
facts_html=facts_html.replace('\n', '')

Scrape 5: Mars Hemispheres Images

In [26]:
# Page url
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# List of hemisphere names
hemispheres=["Valles Marineris Hemisphere","Cerberus Hemisphere","Schiaparelli Hemisphere","Syrtis Major Hemisphere"]

# Initialize list of dictionaries
hemisphere_image_urls=[]

# Loop through web pages & pull attributes
for hemi in hemispheres:
    
    # Visit page
    browser.visit(url)
    
    # Click on hemisphere link
    browser.click_link_by_partial_text(hemi)
    
    # Pull HTML
    html=browser.html
    # Parse HTML with BS4
    soup=bs(html,'html.parser')
    # Pull image element
    img=soup.find('img',class_="wide-image")
    # Create image URL
    img_url='https://astrogeology.usgs.gov'+img['src']
    
    # Create temprorary dictionary
    my_dict={"title":hemi,"img_url":img_url}
    
    # Append dictionary to list
    hemisphere_image_urls.append(my_dict)
    
# Check output
hemisphere_image_urls

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}]

In [21]:
news_result={"headline":headline,"date":date,"paragraph":paragraph}

In [30]:
output={'headline':news_result['headline'],
    'paragraph':news_result['paragraph'],
    'featured_image':featured_url,
    'weather':mars_weather,
    'facts_html':facts_html,
    "Valles Marineris Hemisphere":hemisphere_image_urls[0]["img_url"],
    "Cerberus Hemisphere":hemisphere_image_urls[1]["img_url"],
    "Schiaparelli Hemisphere":hemisphere_image_urls[2]["img_url"],
    "Syrtis Major Hemisphere":hemisphere_image_urls[3]["img_url"]
    }

In [31]:
from flask_pymongo import PyMongo
import scrape_mars

app = Flask(__name__)

# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)

# Or set inline
# mongo = PyMongo(app, uri="mongodb://localhost:27017/craigslist_app")

@app.route("/scrape")
def scraper():
    mars = mongo.db.mars
    mars_data = scrape_mars.scrape()
    mars.update({}, mars_data, upsert=True)
    return redirect("/", code=302)

@app.route("/")
def index():
    mars_info = mongo.db.mars.find_one()
    return render_template("index.html", mars_info=mars_info)

if __name__ == "__main__":
    app.run(debug=True)


{'headline': "Things Are Stacking up for NASA's Mars 2020 Spacecraft",
 'paragraph': 'As the July 2020 launch date inches closer, the next spacecraft headed to the Red Planet is assembled for more testing.',
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01322_hires.jpg',
 'weather': 'InSight sol 141 (2019-04-20) low -98.3ºC (-144.9ºF) high -19.7ºC (-3.5ºF)\nwinds from the SW at 4.7 m/s (10.6 mph) gusting to 12.9 m/s (28.8 mph)\npressure at 7.40 hPa',
 'facts_html': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp